Volume based trend setter

In [4]:
!pip install pandas-ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=12e3af5151f554365d47907c195cb123096f2e3d48117ee7b9d47c6dfeaf9644
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas-ta


In [24]:
# IMPORTS
import pandas as pd
import pandas_ta as ta
import numpy as np
import itertools

In [25]:
#READING AND CLEANING + SET INDEX
df = pd.read_csv("/content/drive/MyDrive/price-data/Binance_BTCUSDT_1min.csv")
df = df.iloc[:,:6]
df.columns=['Date','Open', 'High', 'Low', 'Close', 'Volume']
df.reset_index(drop=True, inplace=True)
df.Date = pd.to_datetime(df.Date)
df = df.set_index("Date")
df

,Open,High,Low,Close,Volume
Date,,,,,
2017-08-17 04:00:00,4261.48,4261.48,4261.48,4261.48,1.775183
2017-08-17 04:01:00,4261.48,4261.48,4261.48,4261.48,0.000000
2017-08-17 04:02:00,4280.56,4280.56,4280.56,4280.56,0.261074
2017-08-17 04:03:00,4261.48,4261.48,4261.48,4261.48,0.012008
2017-08-17 04:04:00,4261.48,4261.48,4261.48,4261.48,0.140796
...,...,...,...,...,...
2023-06-13 16:46:00,25848.59,25852.45,25843.93,25850.01,87.557760
2023-06-13 16:47:00,25850.00,25866.98,25850.00,25866.97,50.890100
2023-06-13 16:48:00,25866.98,25866.98,25855.67,25863.11,53.087330


In [15]:
#MASKING TO DECREASE SAMPLE SIZE
start_date = "2020-01-01 00:00:00"
end_date = "2024-01-01 00:00:00"
mask = (df.index > start_date) & (df.index <= end_date)
df=df.loc[mask]
df

,Open,High,Low,Close,Volume
Date,,,,,
2020-01-01 00:01:00,7187.67,7188.06,7182.20,7184.03,7.248148
2020-01-01 00:02:00,7184.41,7184.71,7180.26,7182.43,11.681677
2020-01-01 00:03:00,7183.83,7188.94,7182.49,7185.94,10.025391
2020-01-01 00:04:00,7185.54,7185.54,7178.64,7179.78,14.911105
2020-01-01 00:05:00,7179.76,7182.51,7178.20,7179.99,12.463243
...,...,...,...,...,...
2023-06-13 16:46:00,25848.59,25852.45,25843.93,25850.01,87.557760
2023-06-13 16:47:00,25850.00,25866.98,25850.00,25866.97,50.890100
2023-06-13 16:48:00,25866.98,25866.98,25855.67,25863.11,53.087330


In [26]:
#Function for resampling the dataframe
def resample_df(df, freq):
    resampled_open = df.Open.resample(freq).first()
    resampled_high = df.High.resample(freq).max()
    resampled_low = df.Low.resample(freq).min()
    resampled_close = df.Close.resample(freq).last()
    resampled_volume = df.Volume.resample(freq).sum()
    new_df = pd.concat([resampled_open, resampled_high, resampled_low, resampled_close, resampled_volume], axis=1)
    new_df.dropna(inplace=True)
    return new_df

In [27]:
#Function for calling open price of next row
def calc_price(new_df):
    new_df["price"] = new_df.Open.shift(-1)

In [34]:
def calc_vol(new_df, x):
  new_df["volma"] = new_df.Volume.rolling(20).mean()

In [38]:
def calc_buy_signal(new_df):
  new_df["buy_signal"] = np.where(((new_df.Volume>new_df.volma*1.5) & (new_df.Close.shift(1)<new_df.Close)), True, False)

In [50]:
def calc_ret(new_df):
  new_df["ret"] = (new_df.Close/new_df.Close.shift(1))-1

In [51]:
new_df = resample_df(df, "1D")
calc_price(new_df)
calc_vol(new_df, 20)
calc_buy_signal(new_df)
calc_ret(new_df)

In [55]:
new_df[new_df.buy_signal]

,Open,High,Low,Close,Volume,price,volma,buy_signal,ret
Date,,,,,,,,,
2017-09-05,4106.97,4484.99,3603.00,4366.47,1228.938157,4366.49,741.012710,True,0.064964
2017-09-15,3188.01,3856.00,2817.00,3700.00,1968.866492,3674.01,880.714683,True,0.160231
2017-09-29,4174.50,4263.86,3952.01,4174.69,1602.309565,4175.00,950.519515,True,0.000046
2017-10-12,4821.43,5439.99,4810.16,5430.00,1276.701482,5430.00,781.241168,True,0.126222
2017-10-13,5430.00,5846.17,5379.84,5649.98,1879.827620,5650.00,837.592909,True,0.040512
...,...,...,...,...,...,...,...,...,...
2023-03-14,24112.27,26386.87,23976.42,24670.41,699360.934230,24670.41,347005.323368,True,0.023096
2023-03-17,24998.78,27756.84,24890.00,27395.13,624460.680910,27395.13,382601.687188,True,0.095859
2023-04-26,28300.80,30036.00,27235.00,28415.29,129228.404030,28415.29,55971.696414,True,0.004046


In [49]:
new_df

,Open,High,Low,Close,Volume,price,volma,buy_signal,ret
Date,,,,,,,,,
2017-08-17,4261.48,4485.39,4200.74,4285.08,795.150377,4285.08,NaN,False,NaN
2017-08-18,4285.08,4371.52,3938.77,4108.37,1199.888264,4108.37,NaN,False,-176.71
2017-08-19,4108.37,4184.69,3850.00,4139.98,381.309763,4139.98,NaN,False,31.61
2017-08-20,4139.98,4211.08,4032.62,4086.29,467.083022,4069.13,NaN,False,-53.69
2017-08-21,4069.13,4119.62,3911.79,4016.00,691.743060,4016.00,NaN,False,-70.29
...,...,...,...,...,...,...,...,...,...
2023-06-09,26498.62,26783.33,26269.91,26477.81,27934.709700,26477.80,37918.147970,False,-20.80
2023-06-10,26477.80,26533.87,25358.00,25841.21,64944.601080,25841.22,40097.984384,False,-636.60
2023-06-11,25841.22,26206.88,25634.70,25925.55,30014.295950,25925.54,40275.757267,False,84.34
